In [1]:
import numpy as np
from Bio import Phylo
from scipy.special import comb
from utilities import *

np.random.seed(42)

In [17]:
class GenSimulator():
    """
    Base class that provide simulation for one genom
    population_size_over_time - times when change lambda
    """
    def __init__(self, number_of_descendants: int, 
                 population_size_over_time: list,
                 mutation_rate: int,
                 lambda_distributation: list):
        self.number_of_descendants = number_of_descendants
        self.lambda_distributation = lambda_distributation
        self.population_size_over_time = population_size_over_time
        self.mutation_rate = mutation_rate
        
        self.poisson_rate = number_of_descendants*mutation_rate + comb(number_of_descendants,2)/lambda_distributation[0]
        
        self.current_population = {}
        self.tree = None
        self.table = {}
    
    
    def init_population(self):
        for i in range(self.number_of_descendants):
            name = generate_name()
            while name in self.current_population.keys(): name = generate_name()
            self.current_population[name] = "{}:{}".format(name, '{}')
            
    def do_event(self, event_type, *args):
        if event_type == 'coalescence':
            self.do_coalescence(*args)
        elif event_type == 'mutation':
            self.do_mutation(*args)
        else:
            raise 'Type not in list'
    
    def do_coalescence(self, current_linage_number, t, delta_t):
        #print('coalescence')
        name_1 = np.random.choice(list(self.current_population.keys()))
        name_2 = np.random.choice(list(self.current_population.keys()))
        while name_1 == name_2: name_2 = np.random.choice(list(self.current_population.keys()))
        left = self.current_population.pop(name_1)
        right = self.current_population.pop(name_2)
        new_name = '{}_{}'.format(name_1, name_2)
        new_branch = '(' + left + ',' + right + ')'
        self.current_population[new_name] =  new_branch.format(delta_t, delta_t)
    
    def do_mutation(self, current_linage_number, t, delta_t):
        #print('mutation')
        position = np.random.uniform()
        while position in self.table.keys(): position = np.random.uniform()
        name = np.random.choice(list(self.current_population.keys()))
        self.table[position] = name
        
    def generate(self):
        self.init_population()
        t = 0
        i = 0
        current_limit = self.population_size_over_time[i]
        current_linage_number = self.number_of_descendants
        
        while current_linage_number > 1:
            #print(len(self.current_population))
            generated_t = np.random.poisson(self.poisson_rate)
            if generated_t > current_limit and i < len(self.population_size_over_time) - 1:
                # Nothing happens in this intervals
                i = i + 1
                t = current_limit
                current_limit = self.population_size_over_time[i]
                self.poisson_rate = current_linage_number*self.mutation_rate + comb(current_linage_number,2)/self.lambda_distributation[i+1]
                continue
            else:
                t = t + generated_t
                # Chose what happens - mutation or coalesent
                s = current_linage_number*self.mutation_rate + comb(current_linage_number,2)/self.lambda_distributation[i+1]
                event_type = np.random.choice(
                    ['mutation', 'coalescence'],
                    p=[current_linage_number*self.mutation_rate/s,
                       (comb(current_linage_number,2)/self.lambda_distributation[i+1])/s]
                )
                self.do_event(event_type, current_linage_number, t, generated_t)
                current_linage_number = len(self.current_population)
                self.poisson_rate = current_linage_number*self.mutation_rate + comb(current_linage_number,2)/self.lambda_distributation[i+1]
        for tree in self.current_population.values():
            self.tree = tree
    
    def __call__(self):
        if self.tree is None:
            raise "Firstly need to be generated"
        return self.tree, self.table
    

In [18]:
lam_time = [6,12]
lam = [3,1,3]
mr = .25

In [19]:
test = GenSimulator(100, lam_time, mr, lam)
test.generate()
test()

100
100
coalescence
99
coalescence
98
coalescence
97
coalescence
96
coalescence
95
coalescence
94
coalescence
93
coalescence
92
coalescence
91
coalescence
90
coalescence
89
coalescence
88
coalescence
87
coalescence
86
coalescence
85
coalescence
84
coalescence
83
coalescence
82
coalescence
81
coalescence
80
coalescence
79
coalescence
78
coalescence
77
coalescence
76
mutation
76
coalescence
75
coalescence
74
coalescence
73
coalescence
72
coalescence
71
coalescence
70
coalescence
69
coalescence
68
coalescence
67
coalescence
66
coalescence
65
coalescence
64
coalescence
63
coalescence
62
coalescence
61
coalescence
60
coalescence
59
coalescence
58
coalescence
57
coalescence
56
coalescence
55
coalescence
54
coalescence
53
coalescence
52
coalescence
51
coalescence
50
coalescence
49
coalescence
48
coalescence
47
coalescence
46
coalescence
45
coalescence
44
coalescence
43
coalescence
42
coalescence
41
mutation
41
coalescence
40
coalescence
39
coalescence
38
coalescence
37
coalescence
36
coalesce

('((((vdeK:39,((((80Sl:544,((ehHV:1022,nrY1:1022),2Soz:679)),((HFSY:1510,7GxM:1510),(LkG9:1198,(OWmH:1363,641J:1363)))),(COjb:486,sKMu:486)),((f68o:174,ehyA:174),((APYX:176,(uplb:1246,D4DU:1246)),(xwoA:147,((wYcP:734,(hsLc:1208,IS2i:1208)),((8VGS:311,(JY8q:327,(((UUdg:1449,5WuT:1449),6emU:805),1u1k:385))),TwDS:269))))))),(k4YX:958,Bx8I:958)),((((58KP:281,Z3lL:281),7KIw:170),(rBIQ:1298,YszV:1298)),((AT1Q:47,(iWCm:256,(((f25J:994,ccBR:994),(zMB5:910,6U9Z:910)),UwWU:597))),hZSq:30))),((((((1jP9:360,(Ykmj:1249,kWWL:1249)),(((1QrJ:308,(A0Hk:395,BNtW:395)),(kSdz:1559,(Ku9W:1675,rKsU:1675))),((((I7rN:1188,dDGJ:1188),lIQa:1161),oOwN:407),(((Gl5Y:1441,qY0n:1441),CvQc:912),RuRk:814)))),(((H3BK:1093,0bCL:1093),((MdYa:785,qWGF:785),i4MB:649)),((SXeD:665,(gdK3:1546,ctfA:1546)),(1SUl:1509,cpDN:1509)))),(((OxI3:1636,5lfq:1636),pXFv:91),(UEA9:39,(3x6c:1117,0DPK:1117)))),(qSFV:561,aEdW:561)),(((SIVi:51,(0L5D:962,oOMb:962)),(((ArBY:684,trrb:684),(3tOT:439,(T8E2:567,RRZF:567))),(kVF5:1230,Eywx:1230))),((

In [114]:
list(a.keys())

['q']